# Testes Iniciais

## Leitura das Credenciais

In [1]:
import pandas as pd

hosts = pd.read_csv('data/host.csv', delimiter=';', index_col='name')

creds = lambda index, col_name : hosts.at[index, col_name]

# print(hosts)

## Parâmetros globais

In [2]:
import logging

logging.basicConfig(level=logging.DEBUG)
_logger = logging.getLogger(__name__)


In [3]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


In [4]:
host_name = 'tkl-odoo16-vm-18'

server = creds(host_name, 'server')
master_pw = creds(host_name, 'master_pw')
dbname = creds(host_name, 'dbname')
user = creds(host_name, 'user')
user_pw = creds(host_name, 'user_pw')
user_apikey = creds(host_name, 'user_apikey')

_logger.info('host_name: "{0}"'.format(host_name))

# print(server_url)


INFO:__main__:host_name: "tkl-odoo16-vm-18"


## Teste de conexão à Base de dados

In [5]:
import erppeek
import xmlrpc.client as xmlrpclib

url = server
sock_common = xmlrpclib.ServerProxy('{}/xmlrpc/2/common'.format(url))
_logger.info('sock_common.version(): "{0}"'.format(sock_common.version()))

client = erppeek.Client(server=server)
_logger.info('Databases found: {0}'.format(client.db.list()))


INFO:__main__:sock_common.version(): "{'server_version': '16.0', 'server_version_info': [16, 0, 0, 'final', 0, ''], 'server_serie': '16.0', 'protocol_version': 1}"
INFO:__main__:Databases found: ['clvhealth_jcafb_16']


## Teste de 'clvsol_odoo_client2.host.host_login'

In [6]:
from clvsol_odoo_client2 import host

host.host_login(server, dbname, user, user_pw)


INFO:clvsol_odoo_client2.host:--> host_login https://192.168.25.212 clvhealth_jcafb_16
INFO:clvsol_odoo_client2.host:--> uid 2
INFO:clvsol_odoo_client2.host:--> sock <ServerProxy for 192.168.25.212/xmlrpc/2/object>
INFO:clvsol_odoo_client2.host:--> [01] Login Ok. Administrator 
INFO:clvsol_odoo_client2.host:--> {'id': 2, 'name': 'Administrator', 'parent_id': False}


(2, <ServerProxy for 192.168.25.212/xmlrpc/2/object>, '[01] Login Ok.')

In [7]:
host.host_login(server + 'x', dbname, user, user_pw)


INFO:clvsol_odoo_client2.host:--> host_login https://192.168.25.212x clvhealth_jcafb_16
INFO:clvsol_odoo_client2.host:--> [11] Server is not responding.  


(False, False, '[11] Server is not responding.')

In [8]:
host.host_login(server, dbname + 'x', user, user_pw)

INFO:clvsol_odoo_client2.host:--> host_login https://192.168.25.212 clvhealth_jcafb_16x
INFO:clvsol_odoo_client2.host:--> except Fault
INFO:clvsol_odoo_client2.host:--> [22] Database does not exist.  


(False, False, '[22] Database does not exist.')

In [9]:
host.host_login(server, dbname, user + 'x', user_pw)


INFO:clvsol_odoo_client2.host:--> host_login https://192.168.25.212 clvhealth_jcafb_16
INFO:clvsol_odoo_client2.host:--> uid False
INFO:clvsol_odoo_client2.host:--> sock <ServerProxy for 192.168.25.212/xmlrpc/2/object>
INFO:clvsol_odoo_client2.host:--> [21] Invalid Login/Pasword.  


(False,
 <ServerProxy for 192.168.25.212/xmlrpc/2/object>,
 '[21] Invalid Login/Pasword.')

In [10]:
host.host_login(server, dbname, user, user_pw + 'x')

INFO:clvsol_odoo_client2.host:--> host_login https://192.168.25.212 clvhealth_jcafb_16
INFO:clvsol_odoo_client2.host:--> uid False
INFO:clvsol_odoo_client2.host:--> sock <ServerProxy for 192.168.25.212/xmlrpc/2/object>
INFO:clvsol_odoo_client2.host:--> [21] Invalid Login/Pasword.  


(False,
 <ServerProxy for 192.168.25.212/xmlrpc/2/object>,
 '[21] Invalid Login/Pasword.')

## Teste de 'CLVhealthJCAFB'

In [15]:
from clvhealth_jcafb import *

clvhealth = CLVhealthJCAFB(server=server,
                           admin_user_pw=user_pw,
                           super_user_pw=master_pw,
                           dbname='teste_odoo'
                          )

# _logger.info('clvhealth_jcafb.server: {0}'.format(clvhealth.server))
# _logger.info('clvhealth_jcafb.dbname: {0}'.format(clvhealth.dbname))
# _logger.info('clvhealth_jcafb.db.server: {0}'.format(clvhealth.db.server))
# _logger.info('clvhealth_jcafb.db.dbname: {0}'.format(clvhealth.db.dbname))

clvhealth.install()


INFO:clvsol_odoo_client2.db:--> sock_common.version(): "{'server_version': '16.0', 'server_version_info': [16, 0, 0, 'final', 0, ''], 'server_serie': '16.0', 'protocol_version': 1}"
INFO:clvsol_odoo_client2.db:--> Databases found: "['clvhealth_jcafb_16', 'teste_odoo']"
INFO:clvsol_odoo_client2.db:--> Database "teste_odoo" already exists.
INFO:clvsol_odoo_client2.db:--> Done.
INFO:clvhealth_jcafb:--> newDB: False


In [12]:
# from clvsol_odoo_client2.db import *
# import create_db

# clvhealth_jcafb = create_db.CLVhealthJCAFB(
#     server=server_url,
#     # super_user_pw=cli.super_user_pw,
#     dbname=db_name,
#     admin_user_pw=password,
#     # demo_user_pw='demo',
#     # data_admin_user_pw=cli.data_admin_user_pw,
#     demo_data=False,
#     upgrade_all=False,
#     modules_to_upgrade=[],
#     lang='pt_BR',
#     tz='America/Sao_Paulo'
# )

# db = DB(
#     server=server_url,
#     super_user_pw=master_pw,
#     admin_user_pw=password,
#     # data_admin_user_pw=cli.data_admin_user_pw,
#     dbname=db_name,
#     demo_data=False,
#     lang='pt_BR',
#     tz='America/Sao_Paulo'
# )

# print('--> Executing create_db.py...\n')

# print('--> Executing create_db()...\n')
# clvhealth_jcafb.create_db()
